# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
csv_path = "../output_files/cities_weather.csv"
cities_weather_df = pd.read_csv(csv_path)

cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Upernavik,72.7868,-56.1549,25.25,50,90,5.57,GL,1631369142
1,Saskylakh,71.9167,114.0833,56.17,67,100,9.10,RU,1631369142
2,Rikitea,-23.1203,-134.9692,73.13,72,13,10.85,PF,1631369143
3,Port,-7.0349,29.7638,83.10,33,73,2.21,CD,1631369144
4,Cabo,-8.2833,-35.0333,80.53,61,88,11.90,BR,1631369145
...,...,...,...,...,...,...,...,...,...
561,Swan,18.2242,121.3791,74.41,99,100,5.82,PH,1631369629
562,Shangrao,28.4532,117.9686,77.85,89,2,3.96,CN,1631369630
563,Mizdah,31.4451,12.9801,100.99,13,11,15.93,LY,1631369631
564,Kyrylivka,46.3759,35.3675,71.31,58,100,12.08,UA,1631369632


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure the gmaps
gmaps.configure(api_key = g_key)

# Create variable to store Latitude and Longitude
city_lat_lng = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)

# Create figure layout options in dictionary
figure_layout = {"width": "100%",
                 "height": "600px",
                 "padding": "1px"}

# Create heat layer
fig = gmaps.figure(layout = figure_layout, center = (20.0, 10.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_lat_lng,
                                 weights = humidity,
                                 max_intensity = np.max(humidity),
                                 dissipating = False,
                                 point_radius = 2)

# add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create dataframe for ideal locations with cities meets the requirements:
# A max temperature between 70(F) - 80(F)
# Wind speed less than 10 mph
# Zero cloudiness
vacation_cities_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] >= 70) &
                                           (cities_weather_df["Max Temp"] <= 80) &
                                           (cities_weather_df["Wind Speed"] <= 10) &
                                           (cities_weather_df["Cloudiness"] == 0)].dropna()

vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,Mar,43.4002,-4.0094,76.14,74,0,8.05,ES,1631369150
66,Rivne,50.6231,26.2274,77.85,31,0,2.24,UA,1631369200
165,Marondera,-18.1853,31.5519,71.44,19,0,8.97,ZW,1631369284
220,West,31.8024,-97.0917,73.51,66,0,5.01,US,1631369331
291,Zaoyang,32.1272,112.7542,77.32,65,0,6.58,CN,1631369230
355,Havelock,34.8791,-76.9013,77.05,74,0,1.99,US,1631369451
525,Avera,33.1940,-82.5271,77.68,47,0,5.17,US,1631369599
526,Beyneu,45.3167,55.2000,77.47,22,0,5.48,KZ,1631369600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Drop un-needed columns from the original data frame
hotel_df = vacation_cities_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

# Base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters for the API call
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# Perform API calls to collect hotel name
for index, row in hotel_df.iterrows():
    
    # Insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    # Store into results part of dictionary
    hotel_results = response["results"]
    
    # Print hotel name and add to data frame if hotel is successfully found
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # Print error statement if hotels cannot be found
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")

hotel_df

Mar, ES hotel: Hotel Suances
Rivne, UA hotel: Ukraine hotel
Marondera, ZW hotel: HopeFay Conference Centre
West, US hotel: Holiday Inn Express & Suites N Waco Area - West
Zaoyang, CN hotel: Hancheng Hotel
Havelock, US hotel: Hampton Inn Havelock
No hotel found for this city within 5000 meters.
Beyneu, KZ hotel: Hotel Бейнеу


,City,Lat,Lng,Country,Hotel Name
57,Mar,43.4002,-4.0094,ES,Hotel Suances
66,Rivne,50.6231,26.2274,UA,Ukraine hotel
165,Marondera,-18.1853,31.5519,ZW,HopeFay Conference Centre
220,West,31.8024,-97.0917,US,Holiday Inn Express & Suites N Waco Area - West
291,Zaoyang,32.1272,112.7542,CN,Hancheng Hotel
355,Havelock,34.8791,-76.9013,US,Hampton Inn Havelock
525,Avera,33.1940,-82.5271,US,
526,Beyneu,45.3167,55.2000,KZ,Hotel Бейнеу


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))